Installing Fake_useragent and Pandas

In [1]:
pip install fake_useragent pandas

  Using cached fake_useragent-2.0.3-py3-none-any.whl.metadata (17 kB)
  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached fake_useragent-2.0.3-py3-none-any.whl (201 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached numpy-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.1 MB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Importing all necessary libraries

In [2]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd

Setting up IMDb Scraper

In [3]:
url = 'https://www.imdb.com/chart/boxoffice/?ref_=nv_ch_cht'
ua = UserAgent()
userAgent = ua.random
headers = {'User-Agent': userAgent}
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, 'html.parser')

Scraping The top 10 movies, thier titles, Numbers and Rate votes

In [4]:
movie_box= soup.find_all('div', class_ = "ipc-metadata-list-summary-item__c")

titles = []
numbers = []
rate_votes =[]
for box in movie_box:

    if box.find('h3', class_ = "ipc-title__text") is not None:
            title = box.find('h3', class_= "ipc-title__text")
            titles.append(title.text)
    else:
        titles.append('None')

    if box.find('ul', attrs={'data-testid' : "title-metadata-box-office-data-container"}) is not None:
            number = box.find('ul', attrs={'data-testid' : "title-metadata-box-office-data-container"})
            numbers.append(number.text)
    else:
        numbers.append('None')

    if box.find('span', class_ = "ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating") is not None:
        rate_vote = box.find('span', class_ ="ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating")
        rate_votes.append(rate_vote.text.replace('\xa0', ' '))
    else:
        rate_votes.append('None')


rate_votes

['5.6 (4.5K)',
 '6.7 (35K)',
 '7.1 (2.5K)',
 '7.0 (37K)',
 '6.9 (70K)',
 '6.7 (4.3K)',
 '5.9 (9.5K)',
 '7.7 (22K)',
 '6.5 (7.3K)',
 '8.0 (14K)']

Saving them into a Panda DataFrame

In [5]:
movie_df = pd.DataFrame({
    'Titles':titles,
    'Numbers': numbers,
    'Rate_Votes': rate_votes
})
movie_df

,Titles,Numbers,Rate_Votes
0,1. Flight Risk,Weekend Gross: $12MTotal Gross: $15MWeeks Rele...,5.6 (4.5K)
1,2. Mufasa: The Lion King,Weekend Gross: $8.6MTotal Gross: $223MWeeks Re...,6.7 (35K)
2,3. One of Them Days,Weekend Gross: $8MTotal Gross: $28MWeeks Relea...,7.1 (2.5K)
3,4. Sonic the Hedgehog 3,Weekend Gross: $5.3MTotal Gross: $227MWeeks Re...,7.0 (37K)
4,5. Moana 2,Weekend Gross: $4.2MTotal Gross: $451MWeeks Re...,6.9 (70K)
5,6. Presence,Weekend Gross: $3.3MTotal Gross: $4.3MWeeks Re...,6.7 (4.3K)
6,7. Wolf Man,Weekend Gross: $3.2MTotal Gross: $19MWeeks Rel...,5.9 (9.5K)
7,8. A Complete Unknown,Weekend Gross: $3.1MTotal Gross: $64MWeeks Rel...,7.7 (22K)
8,9. Den of Thieves 2: Pantera,Weekend Gross: $3MTotal Gross: $33MWeeks Relea...,6.5 (7.3K)
9,10. The Brutalist,Weekend Gross: $2.9MTotal Gross: $9.3MWeeks Re...,8.0 (14K)


Cleaning the dataframe and saving into a csv file

In [6]:
movie_df['Titles'] = movie_df['Titles'].str.replace(r'^\d+\.\s*', '', regex=True)
movie_df['Numbers'] = movie_df['Numbers'].str.replace('Weekend Gross:', '')
movie_df['Numbers'] = movie_df['Numbers'].str.replace('Total Gross: ', '|')
movie_df['Numbers'] = movie_df['Numbers'].str.replace('Weeks Released: ', '|')
movie_df[['Weekend Gross', 'Total Gross', 'Weeks Released']] = movie_df['Numbers'].str.split('|', expand=True)
movie_df[['Rating','Vote_count']] = movie_df['Rate_Votes'].str.split(' ', expand= True)
movie_df.drop(['Numbers', 'Rate_Votes'], axis=1, inplace=True)

movie_df.to_csv('Movie_Data.csv', index=False)
